# 02 — Nettoyage et preparation des données

Objectif :
- Appliquer un nettoyage ciblé et justifié des données Olist
- Préparer des tables propres pour le feature engineering
- Respecter le périmètre défini (commandes livrées, niveau client)

# Import et configuration

In [1]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

PROJECT_DIR = Path.cwd().parent
RAW_DIR = PROJECT_DIR / "data" / "raw"
INTERIM_DIR = PROJECT_DIR / "data" / "interim"

INTERIM_DIR.mkdir(parents=True, exist_ok=True)


# Chargement des tables brutes

In [2]:
customers = pd.read_csv(RAW_DIR / "olist_customers_dataset.csv")
orders = pd.read_csv(RAW_DIR / "olist_orders_dataset.csv")
order_items = pd.read_csv(RAW_DIR / "olist_order_items_dataset.csv")
payments = pd.read_csv(RAW_DIR / "olist_order_payments_dataset.csv")
reviews = pd.read_csv(RAW_DIR / "olist_order_reviews_dataset.csv")
products = pd.read_csv(RAW_DIR / "olist_products_dataset.csv")


# Nettoyage de la table ORDERS - qui est la table clé du projet

In [3]:
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")


# Filtrer les commandes livrés

In [4]:
orders_delivered = orders[orders["order_status"] == "delivered"].copy()

print("Commandes totales :", orders.shape[0])
print("Commandes livrées :", orders_delivered.shape[0])


Commandes totales : 99441
Commandes livrées : 96478


# verification post filtrage 

In [6]:
print(orders_delivered.isna().mean().sort_values(ascending=False))


order_approved_at                0.000145
order_delivered_customer_date    0.000083
order_delivered_carrier_date     0.000021
order_id                         0.000000
order_purchase_timestamp         0.000000
order_status                     0.000000
customer_id                      0.000000
order_estimated_delivery_date    0.000000
dtype: float64


# Nettoyage de la table REVIEWS

### colonne utiles uniquement

In [7]:
reviews_clean = reviews[[
    "order_id",
    "review_score"
]].copy()


In [ ]:
# Valeurs manquantes AVANT nettoyage
reviews.isna().mean().sort_values(ascending=False)


In [11]:
print(reviews_clean.head(10))


                           order_id  review_score
0  73fc7af87114b39712e6da79b0a377eb             4
1  a548910a1c6147796b98fdf73dbeba33             5
2  f9e4b658b201a9f2ecdecbb34bed034b             5
3  658677c97b385a9be170737859d3511b             5
4  8e6bfb81e283fa7e4f11123a3fb894f1             5
5  b18dcdf73be66366873cd26c5724d1dc             1
6  e48aa0d2dcec3a2e87348811bcfdf22b             5
7  c31a859e34e3adac22f376954e19b39d             5
8  9c214ac970e84273583ab523dfafd09b             5
9  b9bf720beb4ab3728760088589c62129             4


Les champs textuels ont été exclus car hors périmètre de l’analyse,
le score numérique étant suffisant pour mesurer la satisfaction client.

In [13]:
print(reviews_clean.isna().mean() * 100)


order_id        0.0
review_score    0.0
dtype: float64


Les avis clients ont été nettoyés en conservant uniquement le score de satisfaction,
permettant une mesure fiable et homogène de l’expérience client.

# Nettoyage de la table PRODUCTS

### Gestion des categories manquantes

In [8]:
products_clean = products.copy()

products_clean["product_category_name"] = (
    products_clean["product_category_name"]
    .fillna("unknown")
)


In [15]:
print(products_clean["product_category_name"].value_counts().head(10))


product_category_name
cama_mesa_banho           3029
esporte_lazer             2867
moveis_decoracao          2657
beleza_saude              2444
utilidades_domesticas     2335
automotivo                1900
informatica_acessorios    1639
brinquedos                1411
relogios_presentes        1329
telefonia                 1134
Name: count, dtype: int64


In [16]:
(products_clean["product_category_name"] == "unknown").mean() * 100


np.float64(1.8512336499650999)

# Sauvegarde des tables nettoyés

In [9]:
customers.to_parquet(INTERIM_DIR / "customers_clean.parquet", index=False)
orders_delivered.to_parquet(INTERIM_DIR / "orders_delivered.parquet", index=False)
order_items.to_parquet(INTERIM_DIR / "order_items_clean.parquet", index=False)
payments.to_parquet(INTERIM_DIR / "payments_clean.parquet", index=False)
reviews_clean.to_parquet(INTERIM_DIR / "reviews_clean.parquet", index=False)
products_clean.to_parquet(INTERIM_DIR / "products_clean.parquet", index=False)

print("Tables nettoyées sauvegardées dans data/interim/")


Tables nettoyées sauvegardées dans data/interim/


Le nettoyage a été réalisé de manière ciblée,
en cohérence avec les objectifs e-commerce du projet.
Seules les anomalies impactant directement
la segmentation client et la modélisation prédictive
ont été traitées, afin de préserver l’information métier.

## Vérification post-nettoyage

- Les commandes livrées représentent l’essentiel de l’activité e-commerce
- Les valeurs manquantes résiduelles sont marginales
- Le nettoyage des avis et des produits a été réalisé sans perte d’information critique
- Les données sont désormais prêtes pour le feature engineering client
